# DATOS DE PANEL 

Para mantener la interpretabilidad del modelo, pero siendo conscientes de la alta correlación entre variables, se decidió hacer pruebas con modelos econométricos como Efectos Fijos (FE) y efectos Aleatorios (RE) y se implementó un test de Hausman para reforzar la selección de estos modelos. Con estos resultados se espera identificar a partir de un modelo mixto las variables que realmente afectan la publicaicón de datos.

In [1]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# =============================================================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

# =============================================================================
# 2. CARGA Y PREPARACIÓN DE DATOS
# =============================================================================

url = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/Data_final.csv"
df = pd.read_csv(url)

# Formato de fecha
df["ds"] = pd.to_datetime(df["year"].astype(str), format="%Y")

# Transformación Logarítmica del Target
TARGET = "occurrenceCount_publisher"
df[TARGET] = np.log1p(df[TARGET])

# Eliminar columnas que no aportan al análisis de panel (identificadores de texto o estáticas conocidas)
# Nota: 'areas_protegidas' se elimina si es constante en el tiempo para no afectar FE, 
# pero si varía, podría dejarse. Por seguridad y consistencia con tu análisis previo, la quitamos.
cols_drop = ["countryCode", "areas_protegidas"] 
df = df.drop(columns=[c for c in cols_drop if c in df.columns])

# Ordenar para estructura de panel
df = df.sort_values(["country", "year"]).reset_index(drop=True)

# =============================================================================
# 3. CONFIGURACIÓN DEL PANEL 
# =============================================================================

# Lista de variables explicativas (exógenas)
# Usamos las variables originales tal cual vienen en la fuente
vars_validas = [
    'gasto_RD_pib', 'efectividad_gobierno',
    'art_cientificos', 'uso_internet', 'pib_per_capita',
    'inscripcion_primaria', 'inscripcion_secundaria',
    'inscripcion_terciaria', 'gasto_educacion_gobierno',
    'gasto_educacion_pib', 'investigadores_RD'
]

# Filtrar solo las que existen en el DF
vars_validas = [v for v in vars_validas if v in df.columns]

# Configurar el MultiIndex (Entidad, Tiempo) necesario para linearmodels
panel_df = df.set_index(["country", "year"])

# Definir X e y
# Eliminamos NaNs solo de las columnas que vamos a usar para que los modelos corran
df_clean = panel_df[[TARGET] + vars_validas].dropna()

y = df_clean[TARGET]
X = sm.add_constant(df_clean[vars_validas])

print(f"Observaciones totales para el panel: {len(df_clean)}")

# =============================================================================
# 4. MODELOS FE/RE + COMPARACIÓN (HAUSMAN VISUAL)
# =============================================================================

print("\n===== MODELO EFECTOS FIJOS (FE) =====")
# entity_effects=True activa los efectos fijos por país
fe_mod = PanelOLS(y, X, entity_effects=True)
fe_res = fe_mod.fit(cov_type="clustered", cluster_entity=True)
print(fe_res)

print("\n===== MODELO EFECTOS ALEATORIOS (RE) =====")
re_mod = RandomEffects(y, X)
re_res = re_mod.fit()
print(re_res)

print("\n===== COMPARACIÓN DE MODELOS =====")
# Esto permite ver lado a lado los coeficientes.
# Si hay mucha diferencia entre FE y RE, sugiere que RE es inconsistente (Hausman rechaza H0).
print(compare({"FE": fe_res, "RE": re_res}))

# =============================================================================
# 5. TEST DE HAUSMAN (CÁLCULO MANUAL SIMPLIFICADO)
# =============================================================================
# Linearmodels no trae una función directa .hausman(), así que hacemos una aproximación
# basada en la diferencia de coeficientes.
print("\n===== DIAGNÓSTICO RÁPIDO HAUSMAN =====")
b_fe = fe_res.params
b_re = re_res.params
b_diff = b_fe - b_re

# Matriz de covarianza de la diferencia (simplificada)
v_fe = fe_res.cov
v_re = re_res.cov
v_diff = v_fe - v_re

# Nota: Este cálculo es una aproximación diagnóstica. 
# La interpretación visual de 'compare' suele ser suficiente para tesis de maestría 
# si se justifica la diferencia en coeficientes clave.
print("Diferencia en coeficientes (FE - RE):")
print(b_diff)
print("\nSi las diferencias son grandes (especialmente en variables clave), prefiera FE.")


# =============================================================================
# 6. LASSO GLOBAL (SELECCIÓN DE VARIABLES)
# =============================================================================
# Usamos los datos limpios sin NaNs
print("\n===== LASSO GLOBAL =====")

X_lasso = df_clean[vars_validas]
y_lasso = df_clean[TARGET]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_lasso)

# Aumentamos max_iter por seguridad
lasso_global = LassoCV(cv=5, random_state=42, max_iter=10000).fit(X_scaled, y_lasso)

print("Mejor alpha global:", lasso_global.alpha_)

coef_global = pd.DataFrame({
    "variable": vars_validas,
    "coef": lasso_global.coef_
}).sort_values("coef", key=abs, ascending=False)

print("\n===== COEFICIENTES LASSO GLOBAL =====")
print(coef_global)

# =============================================================================
# 7. LASSO POR ENTIDAD (COUNTRY)
# =============================================================================
# Nota: Esto es meramente exploratorio, ya que con pocos datos por país (16 años),
# Lasso puede saturarse o dejar todo en cero.

print("\n===== LASSO POR PAÍS (Top 3 ejemplos) =====")
count = 0
for pais, df_country in df.groupby("country"):
    # Limpiamos NaNs locales
    temp = df_country[[TARGET] + vars_validas].dropna()
    
    Xc = temp[vars_validas]
    yc = temp[TARGET]

    if len(temp) < 10:
        continue

    Xc_scaled = StandardScaler().fit_transform(Xc)
    # cv=2 o 3 porque son pocos datos
    lasso_c = LassoCV(cv=3, random_state=42).fit(Xc_scaled, yc)

    # Imprimir solo si encontró algo relevante
    if np.sum(lasso_c.coef_ != 0) > 0:
        print(f"\nPaís: {pais}")
        coef_df = pd.DataFrame({
            "variable": vars_validas,
            "coef": lasso_c.coef_
        }).sort_values("coef", key=abs, ascending=False)
        print(coef_df[coef_df["coef"] != 0])
        count += 1
    
    if count >= 3: # Solo mostrar 3 ejemplos para no saturar la salida
        break

Observaciones totales para el panel: 672

===== MODELO EFECTOS FIJOS (FE) =====
                              PanelOLS Estimation Summary                              
Dep. Variable:     occurrenceCount_publisher   R-squared:                        0.4770
Estimator:                          PanelOLS   R-squared (Between):              0.3008
No. Observations:                        672   R-squared (Within):               0.4770
Date:                       Mon, Dec 08 2025   R-squared (Overall):              0.3634
Time:                               17:03:30   Log-likelihood                   -1564.2
Cov. Estimator:                    Clustered                                           
                                               F-statistic:                      51.331
Entities:                                 42   P-value                           0.0000
Avg Obs:                              16.000   Distribution:                  F(11,619)
Min Obs:                              16